In [ ]:
import weakref
import fastplotlib as fpl
import numpy as np
import pytest

In [ ]:
def test_references(plot_objects):
    for i in range(len(plot_objects)):
        with pytest.raises(ReferenceError) as failure:
            plot_objects[i]
            pytest.fail(f"GC failed for object: {plot_objects[i]} of type: {plot_objects[i].__class__.__name__}")

# Add graphics and selectors, add feature event handlers, test gc occurs

In [ ]:
xs = np.linspace(0, 20 * np.pi, 1_000)
ys = np.sin(xs)
zs = np.zeros(xs.size)

points_data = np.column_stack([xs, ys, zs])

line_collection_data = [points_data[:, 1].copy() for i in range(10)]

img_data = np.random.rand(1_000, 1_000)

# Make references to each graphic

In [ ]:
fig = fpl.Figure((2, 2))

line = fig[0, 0].add_line(points_data, name="line")
scatter = fig[0, 1].add_scatter(points_data.copy(), name="scatter")
line_stack = fig[1, 0].add_line_stack(line_collection_data, name="line-stack")
image = fig[1, 1].add_image(img_data, name="image")

linear_sel = line.add_linear_selector(name="line_linear_sel")
linear_region_sel = line.add_linear_region_selector(name="line_region_sel")

linear_sel2 = line_stack.add_linear_selector(name="line-stack_linear_sel")
linear_region_sel2 = line_stack.add_linear_region_selector(name="line-stack_region_sel")

linear_sel_img = image.add_linear_selector(name="image_linear_sel")
linear_region_sel_img = image.add_linear_region_selector(name="image_linear_region_sel")

# Add event handlers

In [ ]:
def feature_changed_handler(ev):
    pass

In [ ]:
objects = list()
weakrefs = list() # used to make sure the real objs are garbage collected
for subplot in fig:
    for obj in subplot.objects:
        objects.append(obj)
        weakrefs.append(weakref.proxy(obj))

for g in objects:
    for feature in g._features:
        if getattr(g, f"_{feature}") is None:
            continue  # not in the right mode to support this feature
        g.add_event_handler(feature_changed_handler, feature)

# Show figure

In [ ]:
fig.show()

# Clear fig and user-created objects list

In [ ]:
fig.clear()

In [ ]:
objects.clear()

# test gc

In [ ]:
test_references(weakrefs)

# test for ImageWidget

In [ ]:
if fpl.IMGUI:
    # do image widget tests only if imgui is installed
    movies = [np.random.rand(100, 100, 100) for i in range(6)]
    
    iw = fpl.ImageWidget(movies)
    
    # add some events onto all the image graphics
    for g in iw.managed_graphics:
        for f in g._features:
            g.add_event_handler(feature_changed_handler, f)
    
    iw.show()
    
    old_graphics = [weakref.proxy(g) for g in iw.managed_graphics]
    
    # Test that setting new data with different dims clears old ImageGraphics
    new_movies = [np.random.rand(100, 200, 200) for i in range(6)]
    
    iw.set_data(new_movies)
    test_references(old_graphics)